# Music Genre Classification using multiple classifiers
Team Members: Lisa Korntheuer, Jan Birkert, Adrian Desiderato, Jan Wangerin, Spyridon Spyropoulos

## 1. Data preparation

## 2. Model training 

Each Modell is trained and the quality of the classifier is displayed. 

### 2.1 Random Forests

### 2.2 Decision trees

### 2.3 Knn 

### 2.4 Linear Regression

### 2.5 Neural Networks

### 2.6 K-Means

### 2.7 Bayes 

## 3. Comparing Models 